In [ ]:
# for fim episodes
MLP_EPISODES_DIRECTORIES = [
    "D:\MLP_Samples\AIData\PPP Cleans"
]
#EPISODE_EXTENSIONS = ['mp4', 'mkv']
EPISODE_EXTENSIONS = ['flac']
EPISODE_PATTERNS = [r's(\d{2})e(\d{2})', r'S(\d{2})E(\d{2})', r'(\d{2})x(\d{2})']

import os
import re

episode_index = {}
for d in MLP_EPISODES_DIRECTORIES:
    for root,_,files in os.walk(d):
        for f in files:
            f_ext = f.split('.')[-1]
            if not f_ext in EPISODE_EXTENSIONS:
                continue
            for pattern in EPISODE_PATTERNS:
                match_s = re.search(pattern, f)
                if match_s:
                    sig = f's{match_s.group(1)}e{match_s.group(2)}'
                    episode_index[sig] = os.path.join(root,f).replace('\\','/')
                
import subprocess
from pydub import AudioSegment
import io
def extract_center_channel(vid_path):
    #command = ['ffmpeg', '-i', vid_path, '-filter_complex',
    #    'channelsplit=channel_layout=5.1:channels=FC[FC]', '-map', '[FC]',
    #    '-c:a', 'pcm_s32le',
    #     '-f', 'wav', '-']
    #process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #output, errs = process.communicate()
    #f = io.BytesIO(output)
    #audio_segment = AudioSegment.from_file(f, format='wav')
    audio_segment = AudioSegment.from_file(vid_path)
    return audio_segment

from demucs import pretrained
from demucs.apply import apply_model
import numpy as np
from pathlib import Path
import torch
import scipy.io.wavfile
class DemucsModels:
    def __init__(self, repo, models):
        if type(repo) == str:
            repo = Path(repo)
        self.models = []
        self.model_names = []
        for model in models:
            self.models.append(pretrained.get_model(name=model, repo=repo))
            self.model_names.append(model)

    def apply(self, x):
        outputs = []
        for model in self.models:
            out = apply_model(model, torch.from_numpy(x).T.unsqueeze(0),
                device='cuda')[0]
            out_dict = {}
            for name, source in zip(model.sources, out):
                out_dict[name] = source
            vocals = out_dict['vocals'].numpy().T
            outputs.append(vocals)
        return outputs

    def process_ep(self, specifier='s01e01', dump_to_folder=None):
        assert specifier in episode_index
        if os.path.exists(dump_to_folder):
            print(f"Skipping folder {dump_to_folder}")
            return
        if dump_to_folder is not None:
            os.makedirs(dump_to_folder, exist_ok=True)

        print(f"processing {specifier}")
        print("extracting center")
        fc = extract_center_channel(episode_index[specifier])
        sr = fc.frame_rate
        fc_dbfs = fc.dBFS
        fc_orig = fc
        fc = np.array(fc.get_array_of_samples())
        fc = np.column_stack((fc,fc))
        fc = (fc / (2 ** 32 - 1)).astype(np.float32)

        print("applying demucs")
        demucs_outputs = self.apply(fc)
        print("finished applying demucs")
        for i,do in enumerate(demucs_outputs):
            do_io = io.BytesIO()
            scipy.io.wavfile.write(do_io, 48000, do)
            do_io.seek(0)
            do_snd = AudioSegment.from_wav(do_io).set_channels(1)
            print(f'dbfs adjust {fc_dbfs - do_snd.dBFS}')
            do_snd = do_snd.apply_gain(fc_dbfs - do_snd.dBFS)
            if dump_to_folder is not None:
                do_snd.export(
                    os.path.join(dump_to_folder,f'{specifier}_demu{i}.flac'), format='flac')

        if dump_to_folder is not None:
            # These are more for previews than anything
            fc_orig.export(
                os.path.join(dump_to_folder,f'{specifier}.flac'),
                format='flac')
        print('done')

demucsmodels = DemucsModels(repo="D:/Code/demucs/release_models",
    models=["SFX_separation_epoch_190", "SFX_separation_v2_epoch_338"])
for k in episode_index.keys():
    dump_folder = os.path.join('episode_process_dumps',k)
    demucsmodels.process_ep(specifier=k, dump_to_folder=dump_folder)


# Objective: Compare 0. original (i.e. Clipper's choice), 1. sfx demucs v1, 2. sfx demucs v2

In [2]:

import os
import re

# Manual mapping
episode_index = {
    'eqg_dance magic' : 'D:/MLP_Samples/AIData/for vul/Dance Magic English.flac',
    'eqg_forgotten friendship' : 'D:/MLP_Samples/AIData/for vul/Forgotten Friendship.flac',
    'eqg_friendship_games' : 'D:/MLP_Samples/AIData/for vul/Friendship Games.flac',
    'eqg_legend_of_everfree' : 'D:/MLP_Samples/AIData/for vul/Legend of Everfree.flac',
    'eqg_mirror magic' : 'D:/MLP_Samples/AIData/for vul/Mirror Magic English.flac',
    'eqg_movie magic' : 'D:/MLP_Samples/AIData/for vul/Movie Magic English.flac',
    'eqg_rollercoaster of friendship' : 'D:/MLP_Samples/AIData/for vul/Rollercoaster of Friendship.flac',
    'eqg_better together_s02e04' : 'D:/MLP_Samples/AIData/for vul/Better Together/S2/EQG Short 204 Street Magic With Trixie.flac',
    'eqg_better together_s02e05' : 'D:/MLP_Samples/AIData/for vul/Better Together/S2/EQG Short 205 Sic Skateboard.flac',
    'eqg_better together_s02e06' : 'D:/MLP_Samples/AIData/for vul/Better Together/S2/EQG Short 206 Street Chic.flac',
    'eqg_better together_s02e07' : 'D:/MLP_Samples/AIData/for vul/Better Together/S2/EQG Short 207 Game Stream.flac',
    'eqg_better together_s02e08' : 'D:/MLP_Samples/AIData/for vul/Better Together/S2/EQG Short 208 Best in Show The Preshow.flac',
    'fim_movie' : 'D:/MLP_Samples/AIData/for vul/MLP Movie_C.flac',
    'fim_rainbow roadtrip' : 'D:/MLP_Samples/AIData/for vul/Rainbow Roadtrip.flac',
}
                
import subprocess
from pydub import AudioSegment
import io
def extract_center_channel(vid_path):
    #command = ['ffmpeg', '-i', vid_path, '-filter_complex',
    #    'channelsplit=channel_layout=5.1:channels=FC[FC]', '-map', '[FC]',
    #    '-c:a', 'pcm_s32le',
    #     '-f', 'wav', '-']
    #process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #output, errs = process.communicate()
    #f = io.BytesIO(output)
    #audio_segment = AudioSegment.from_file(f, format='wav')
    audio_segment = AudioSegment.from_file(vid_path)
    return audio_segment

from demucs import pretrained
from demucs.apply import apply_model
import numpy as np
from pathlib import Path
import torch
import scipy.io.wavfile
class DemucsModels:
    def __init__(self, repo, models):
        if type(repo) == str:
            repo = Path(repo)
        self.models = []
        self.model_names = []
        for model in models:
            self.models.append(pretrained.get_model(name=model, repo=repo))
            self.model_names.append(model)

    def apply(self, x):
        outputs = []
        for model in self.models:
            out = apply_model(model, torch.from_numpy(x).T.unsqueeze(0),
                device='cuda')[0]
            out_dict = {}
            for name, source in zip(model.sources, out):
                out_dict[name] = source
            vocals = out_dict['vocals'].numpy().T
            outputs.append(vocals)
        return outputs

    def process_ep(self, specifier='s01e01', dump_to_folder=None,
        force_16_bit = False):
        assert specifier in episode_index
        if os.path.exists(dump_to_folder):
            print(f"Skipping folder {dump_to_folder}")
            return
        if dump_to_folder is not None:
            os.makedirs(dump_to_folder, exist_ok=True)

        print(f"processing {specifier}")
        print("extracting center")
        fc = extract_center_channel(episode_index[specifier])
        fc = fc.set_channels(1)

        sr = fc.frame_rate
        fc_dbfs = fc.dBFS
        fc_orig = fc
        fc = np.array(fc.get_array_of_samples())
        fc = np.column_stack((fc,fc))
        fc = (fc / (2 ** (16 if force_16_bit else 32) - 1)).astype(np.float32)

        print("applying demucs")
        demucs_outputs = self.apply(fc)
        print("finished applying demucs")
        for i,do in enumerate(demucs_outputs):

            # Stereo to average by mean
            do = np.mean(do, axis=1)

            # pydub can't handle long files.
            def scale_to_dBFS(data, target_dBFS):
                # Calculate the current dBFS level of the data
                rms = np.sqrt(np.mean(data**2))
                current_dBFS = 20 * np.log10(rms)

                # Calculate the scaling factor
                scale_factor = 10 ** ((target_dBFS - current_dBFS) / 20)

                # Scale the data
                scaled_data = data * scale_factor

                return scaled_data
            
            print('fc dbfs',fc_dbfs)
            # Scale to match dBFS
            do = scale_to_dBFS(do, fc_dbfs)
            print("do mean",do.mean())
            print("fc mean",fc.mean())
            
            # No python library can reliably write FLACs, so we use WAV instead then convert to ffmpeg
            scipy.io.wavfile.write("temp.wav", 48000, do)

            import ffmpeg
            ffmpeg.input("temp.wav").output(
                os.path.join(dump_to_folder,f'{specifier}_demu{i}.flac')).run()

        if dump_to_folder is not None:
            # These are more for previews than anything
            fc_orig.export(
                os.path.join(dump_to_folder,f'{specifier}.flac'),
                format='flac')
        print('done')

demucsmodels = DemucsModels(repo="D:/Code/demucs/release_models",
    models=["SFX_separation_epoch_190", "SFX_separation_v2_epoch_338"])
#demucsmodels = DemucsModels(repo="D:/Code/demucs/release_models",
    #models=["SFX_separation_epoch_190"])
for k in episode_index.keys():
    dump_folder = os.path.join('extra_process_dumps',k)
    demucsmodels.process_ep(specifier=k, dump_to_folder=dump_folder)
    # force_16_bit needed for better together


# Objective: Compare 0. original (i.e. Clipper's choice), 1. sfx demucs v1, 2. sfx demucs v2

Skipping folder extra_process_dumps\eqg_dance magic
Skipping folder extra_process_dumps\eqg_forgotten friendship
Skipping folder extra_process_dumps\eqg_friendship_games
Skipping folder extra_process_dumps\eqg_legend_of_everfree
Skipping folder extra_process_dumps\eqg_mirror magic
Skipping folder extra_process_dumps\eqg_movie magic
Skipping folder extra_process_dumps\eqg_rollercoaster of friendship
Skipping folder extra_process_dumps\eqg_better together_s02e04
Skipping folder extra_process_dumps\eqg_better together_s02e05
Skipping folder extra_process_dumps\eqg_better together_s02e06
Skipping folder extra_process_dumps\eqg_better together_s02e07
Skipping folder extra_process_dumps\eqg_better together_s02e08
processing fim_movie
extracting center
applying demucs
finished applying demucs
fc dbfs -27.466810166422743
do mean 2.3391573e-05
fc mean -2.2629613e-06
fc dbfs -27.466810166422743
do mean -7.952566e-06
fc mean -2.2629613e-06
done
Skipping folder extra_process_dumps\fim_rainbow road